In [2]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
#import scipy.stats as stats
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest


# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("world_data_full_apply_nulos.csv", index_col = 0)
df.head(2)

,country,density,abbreviation,agricultural_land,land_area,armed_forces_size,birth_rate,calling_code,capital/major_city,co2-emissions,cpi,cpi_change,currency-code,fertility_rate,forested_area,gasoline_price,gdp,gross_primary_education_enrollment,gross_tertiary_education_enrollment,infant_mortality,largest_city,life_expectancy,maternal_mortality_ratio,minimum_wage,official_language,out_of_pocket_health_expenditure,physicians_per_thousand,population,population_labor_force_participation,tax_revenue,total_tax_rate,unemployment_rate,urban_population,latitude,longitude,continent
0,Afghanistan,60.0,AF,58.1,652.230,323.0,32.49,93.0,Kabul,8.672,149.90,2.3,AFN,4.47,2.1,0.70,1.910135e+10,104.0,9.7,47.9,Kabul,64.5,638.0,0.43,Pashto,78.4,0.28,NaN,48.9,9.3,71.4,11.12,NaN,33.939110,67.709953,Asia
1,Albania,105.0,AL,43.1,28.748,9.0,11.78,355.0,Tirana,4.536,119.05,1.4,ALL,1.62,28.1,1.36,1.527808e+10,107.0,55.0,7.8,Tirana,78.5,15.0,1.12,Albanian,56.9,1.20,NaN,55.7,18.6,36.6,12.33,NaN,41.153332,20.168331,Europe


1. Ejercicios intervalo de confianza. Al igual que en el caso del ejercicio de la clase, piensa un motivo por el que sería interesante calcular estos intervalos de confianza para la extracción de insights.

- A. Intervalo de Confianza para la Esperanza de Vida. Contexto: Trabajas en una organización de salud global y deseas estimar el intervalo de confianza al 95% para la esperanza de vida promedio de cada continente. Nota: Cuidado porque puede que os salgan algunos nulos, genera un código para que no salgan nulos en los intervalos de confianza.


In [ ]:
# intervalo para una columna:
def intervalo_confianza(df, columna, nivel_confianza=0.95):
    media = df[columna].mean()
    error_estandar = stats.sem(df[columna])
    grados_libertad = len(df[columna]) - 1
    valor_critico = stats.t.ppf((1 + nivel_confianza) / 2, df=grados_libertad)
    distancia = valor_critico * error_estandar
    limite_inf = media - distancia
    limite_sup = media + distancia
    
    print(f"Nuestro intervalo de confianza para la columna '{columna}' es de {np.round(limite_inf, 2)} a {np.round(limite_sup, 2)} con un nivel de confianza de {nivel_confianza}")
    print(f"La media es {media}")
    
    return (limite_inf, limite_sup)

In [7]:
intervalo_confianza(df, 'density', nivel_confianza=0.95)

Nuestro intervalo de confianza para la columna 'density' es de 112.29 a 156.72 con un nivel de confianza de 0.95
La media es 134.50453333333334


(np.float64(112.2869946320454), np.float64(156.72207203462128))

In [8]:
# esperanza de vida 'life_expectancy' promedio de cada continente 'continent'
df.columns

Index(['country', 'density', 'abbreviation', 'agricultural_land', 'land_area',
       'armed_forces_size', 'birth_rate', 'calling_code', 'capital/major_city',
       'co2-emissions', 'cpi', 'cpi_change', 'currency-code', 'fertility_rate',
       'forested_area', 'gasoline_price', 'gdp',
       'gross_primary_education_enrollment',
       'gross_tertiary_education_enrollment', 'infant_mortality',
       'largest_city', 'life_expectancy', 'maternal_mortality_ratio',
       'minimum_wage', 'official_language', 'out_of_pocket_health_expenditure',
       'physicians_per_thousand', 'population',
       'population_labor_force_participation', 'tax_revenue', 'total_tax_rate',
       'unemployment_rate', 'urban_population', 'latitude', 'longitude',
       'continent'],
      dtype='object')

In [ ]:
df.groupby('continent')['life_expectancy'].mean()

continent
Africa             63.725000
Asia               73.781818
Central America    74.690000
Europe             79.202326
North America      80.200000
Oceania            71.125000
South America      74.815385
Name: life_expectancy, dtype: float64

In [35]:
# dropna: "elimina" los nulos en la nueva df, no cuenta con ellos para la revisión
data = df.dropna(subset=['life_expectancy', 'continent'])

In [ ]:
# primera prueba del intervalo de 2 columnas. Sale pero no está en orden los prints, lo dejo para saber cómo hemos sacado el siguiente
def intervalo_confianza(data, columna1, columna2, nivel_confianza=0.95):

    medias_continentes = data.groupby(columna1)[columna2].mean()
    for cont in medias_continentes:
        error_estandar = stats.sem(data[columna2])
        grados_libertad = len(data[columna2]) - 1
        valor_critico = stats.t.ppf((1 + nivel_confianza) / 2, df=grados_libertad)
        distancia = valor_critico * error_estandar
        limite_inf = medias_continentes - distancia
        limite_sup = medias_continentes + distancia
        
        print(f"Para el continente {cont} tenemos:")
        print(f"Nuestro intervalo de confianza para la columna '{columna2}' es de {np.round(limite_inf, 2)} a {np.round(limite_sup, 2)} con un nivel de confianza de {nivel_confianza}")
        print(f"La media es {medias_continentes}")
        
        return (limite_inf, limite_sup)

In [27]:
intervalo_confianza(data, 'continent','life_expectancy', nivel_confianza=0.95)

Para el continente 63.724999999999994 tenemos:
Nuestro intervalo de confianza para la columna 'life_expectancy' es de continent
Africa             62.65
Asia               72.70
Central America    73.61
Europe             78.12
North America      79.12
Oceania            70.05
South America      73.74
Name: life_expectancy, dtype: float64 a continent
Africa             64.80
Asia               74.86
Central America    75.77
Europe             80.28
North America      81.28
Oceania            72.20
South America      75.90
Name: life_expectancy, dtype: float64 con un nivel de confianza de 0.95
La media es continent
Africa             63.725000
Asia               73.781818
Central America    74.690000
Europe             79.202326
North America      80.200000
Oceania            71.125000
South America      74.815385
Name: life_expectancy, dtype: float64


(continent
 Africa             62.645366
 Asia               72.702185
 Central America    73.610366
 Europe             78.122692
 North America      79.120366
 Oceania            70.045366
 South America      73.735751
 Name: life_expectancy, dtype: float64,
 continent
 Africa             64.804634
 Asia               74.861452
 Central America    75.769634
 Europe             80.281959
 North America      81.279634
 Oceania            72.204634
 South America      75.895018
 Name: life_expectancy, dtype: float64)

In [ ]:
# ESTE ES EL BUENO!
def intervalo_confianza_esperanza_vida(data, columna1, columna2, nivel_confianza=0.95):
    continentes = sorted(data[columna1].unique())  # Ordenar alfabéticamente los contienentes
    
    for cont in continentes:
        subset = data[data[columna1] == cont][columna2] # Subset: hace una agrupación por cada contienente
        media = subset.mean()
        error_estandar = stats.sem(subset)
        grados_libertad = len(subset) - 1
        
        valor_critico = stats.t.ppf((1 + nivel_confianza) / 2, df=grados_libertad)
        distancia = valor_critico * error_estandar
        limite_inf = media - distancia
        limite_sup = media + distancia
        
        print(f"🌍 Para el continente '{cont}':")
        print(f"   Media de '{columna2}': {np.round(media, 2)}")
        print(f"   Intervalo de confianza: [{np.round(limite_inf, 2)}, {np.round(limite_sup, 2)}] con un nivel de confianza de {nivel_confianza}")
        print("-" * 60)

In [39]:
intervalo_confianza_esperanza_vida(data, 'continent','life_expectancy', nivel_confianza=0.95)

🌍 Para el continente 'Africa':
   Media de 'life_expectancy': 63.72
   Intervalo de confianza: [62.05, 65.4] con un nivel de confianza de 0.95
------------------------------------------------------------
🌍 Para el continente 'Asia':
   Media de 'life_expectancy': 73.78
   Intervalo de confianza: [72.32, 75.24] con un nivel de confianza de 0.95
------------------------------------------------------------
🌍 Para el continente 'Central America':
   Media de 'life_expectancy': 74.69
   Intervalo de confianza: [73.04, 76.34] con un nivel de confianza de 0.95
------------------------------------------------------------
🌍 Para el continente 'Europe':
   Media de 'life_expectancy': 79.2
   Intervalo de confianza: [78.09, 80.32] con un nivel de confianza de 0.95
------------------------------------------------------------
🌍 Para el continente 'North America':
   Media de 'life_expectancy': 80.2
   Intervalo de confianza: [58.6, 101.8] con un nivel de confianza de 0.95
--------------------------

1. Ejercicios intervalo de confianza:
- B. Intervalo de Confianza para el PIB. Contexto: Eres una economista y quieres calcular el intervalo de confianza al 90% para el Producto Interno Bruto (PIB) de los continentes.

In [45]:
df.groupby('continent')['gdp'].mean()

continent
Africa             4.608056e+10
Asia               8.371953e+11
Central America    8.847993e+10
Europe             4.867482e+11
North America      1.158206e+13
Oceania            1.167886e+11
South America      2.984337e+11
Name: gdp, dtype: float64

In [40]:
def intervalo_confianza_PIB(data, columna1, columna2, nivel_confianza=0.90):
    continentes = sorted(data[columna1].unique())  # Ordenar alfabéticamente los contienentes
    
    for cont in continentes:
        subset = data[data[columna1] == cont][columna2] # Subset: hace una agrupación por cada contienente
        media = subset.mean()
        error_estandar = stats.sem(subset)
        grados_libertad = len(subset) - 1
        
        valor_critico = stats.t.ppf((1 + nivel_confianza) / 2, df=grados_libertad)
        distancia = valor_critico * error_estandar
        limite_inf = media - distancia
        limite_sup = media + distancia
        
        print(f"🌍 Para el continente '{cont}':")
        print(f"   Media de '{columna2}': {np.round(media, 2)}")
        print(f"   Intervalo de confianza: [{np.round(limite_inf, 2)}, {np.round(limite_sup, 2)}] con un nivel de confianza de {nivel_confianza}")
        print("-" * 60)

In [42]:
intervalo_confianza_PIB(data, 'continent','gdp', nivel_confianza=0.90)

🌍 Para el continente 'Africa':
   Media de 'gdp': 46893814043.79
   Intervalo de confianza: [26357148610.54, 67430479477.03] con un nivel de confianza de 0.9
------------------------------------------------------------
🌍 Para el continente 'Asia':
   Media de 'gdp': 837195265650.86
   Intervalo de confianza: [58578264502.59, 1615812266799.14] con un nivel de confianza de 0.9
------------------------------------------------------------
🌍 Para el continente 'Central America':
   Media de 'gdp': 88479927703.1
   Intervalo de confianza: [-18731287288.44, 195691142694.64] con un nivel de confianza de 0.9
------------------------------------------------------------
🌍 Para el continente 'Europe':
   Media de 'gdp': 520229234203.4
   Intervalo de confianza: [299218033686.15, 741240434720.64] con un nivel de confianza de 0.9
------------------------------------------------------------
🌍 Para el continente 'North America':
   Media de 'gdp': 11582062814760.0
   Intervalo de confianza: [-50580843